In [ ]:
!pip install datasets transformers

In [ ]:
from datasets import load_dataset
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



#### 데이터셋 확인

In [ ]:
# 데이터셋 로드
dataset = load_dataset("kjchoi/news_summ-data")

# 데이터셋을 Pandas 데이터프레임으로 변환
df = pd.DataFrame(dataset["train"])

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
# 1500자 이상 필터링
count_over_1500 = (df["contents"].str.len() >= 1500).sum()

# 결과 출력
print(f"1500자 이상의 뉴스 개수: {count_over_1500}")

1500자 이상의 뉴스 개수: 19692


#### 뉴스 길이별로 구분

In [ ]:
# 문자 길이 계산
df["char_length"] = df["contents"].str.len()

# 긴 뉴스 (1500자 이상)
long_news = df[df["char_length"] > 1500]

# 중간 뉴스 (900 ~ 1500자)
medium_news_candidates = df[(df["char_length"] >= 900) & (df["char_length"] <= 1500)]

# 짧은 뉴스 (900자 이하)
short_news_candidates = df[df["char_length"] < 900]

# 랜덤 샘플링 (중간 뉴스와 짧은 뉴스에서 각각 20000개)
medium_news_sample = medium_news_candidates.sample(n=min(20000, len(medium_news_candidates)), random_state=42)
short_news_sample = short_news_candidates.sample(n=min(20000, len(short_news_candidates)), random_state=42)

In [ ]:
# 각 데이터에 'length' 열 추가
long_news["length"] = "long"
medium_news_sample["length"] = "medium"
short_news_sample["length"] = "short"

# 데이터 병합
final_df = pd.concat([long_news, medium_news_sample, short_news_sample], ignore_index=True)

<ipython-input-47-4ac11da7006f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_news["length"] = "long"


In [ ]:
# 문자열로 고정
final_df["summary"] = final_df["summary"].astype(str)
final_df["contents"] = final_df["contents"].astype(str)

In [ ]:
final_df.head()

,contents,summary,char_length,length
0,"신년 기자간담회…개발 자제, 시민 행복 정책 초점 이성훈 sinawi@hanmail...","광양시는 2018년 시민의 행복을 위한 도시 조성, 문화·관광기반 조성 등 시민 삶...",1770,long
1,내 인생 첫 스크린‘광양 제일극장’ 광양뉴스 webmaster@gynet.co.kr...,80년대 초반 광양 제일극장은 동시대를 살아온 사람들의 추억이 엮여있는 문화의 공간...,1874,long
2,운영주체 못 찾아…규모 축소 또는 건립 포기 이성훈 sinawi@hanmail.ne...,광양시가 추진하고 있는 도선국사 사상 수련관 조성 사업은 2019년 완공할 계획이었...,1645,long
3,윤동주·정병욱 문화콘텐츠 발굴‘국제학술 심포지엄’ 이성훈 sinawi@hanmail...,지난달 27일 열린 윤동주·정병욱 문화콘츠 발굴 국제학술 심포지엄에서 발표자들은 윤...,1506,long
4,"녹지사업소? 도로과, 서로 떠넘기기‘빈축’ 김영신 기자 yskim0966@naver...",광양읍 한 가로수 아래 보도블럭이 심하게 들떠있어 행인들의 보행안전을 위협하고 도시...,1871,long


In [ ]:
print(final_df["length"].value_counts())

length
medium    20000
short     20000
long      19624
Name: count, dtype: int64


#### train, validation, test 분할

In [ ]:
train_df = pd.DataFrame()
val_df = pd.DataFrame()
test_df = pd.DataFrame()

# 각 길이 구분 별로 데이터셋 분할
for category in ["short", "medium", "long"]:
    category_data = final_df[final_df["length"] == category]
    train, temp = train_test_split(category_data, test_size=0.2, random_state=42)
    val, test = train_test_split(temp, test_size=0.5, random_state=42)

    train_df = pd.concat([train_df, train], ignore_index=True)
    val_df = pd.concat([val_df, val], ignore_index=True)
    test_df = pd.concat([test_df, test], ignore_index=True)

# 결과 확인
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

Train size: 47699
Validation size: 5962
Test size: 5963


In [ ]:
# train
print("Train 데이터 분포:")
print(train_df["length"].value_counts())

# validation
print("\nValidation 데이터 분포:")
print(val_df["length"].value_counts())

# test
print("\nTest 데이터 분포:")
print(test_df["length"].value_counts())

Train 데이터 분포:
length
short     16000
medium    16000
long      15699
Name: count, dtype: int64

Validation 데이터 분포:
length
short     2000
medium    2000
long      1962
Name: count, dtype: int64

Test 데이터 분포:
length
short     2000
medium    2000
long      1963
Name: count, dtype: int64


In [ ]:
# test 데이터셋에서 길이별로 분리
long_test = test_df[test_df["length"] == "long"]
medium_test = test_df[test_df["length"] == "medium"]
short_test = test_df[test_df["length"] == "short"]

In [ ]:
# csv 파일 저장
output_dir = "/content/drive/MyDrive/응용자연어처리/project/data"

train_df.to_csv(os.path.join(output_dir, "train.csv"), index=False, encoding="utf-8-sig")
val_df.to_csv(os.path.join(output_dir, "validation.csv"), index=False, encoding="utf-8-sig")
long_test.to_csv(os.path.join(output_dir, "long_test.csv"), index=False, encoding="utf-8-sig")
medium_test.to_csv(os.path.join(output_dir, "medium_test.csv"), index=False, encoding="utf-8-sig")
short_test.to_csv(os.path.join(output_dir, "short_test.csv"), index=False, encoding="utf-8-sig")

In [ ]:
# train과 validaion에 null 값이 생기는 문제 해결
train_path = "/content/drive/MyDrive/응용자연어처리/project/data/train.csv"
train_df = pd.read_csv(train_path)

print(f"삭제 전 데이터 크기: {train_df.shape}")
train_df = train_df.dropna(subset=["summary"])
print(f"삭제 후 데이터 크기: {train_df.shape}")

validation_path = "/content/drive/MyDrive/응용자연어처리/project/data/validation.csv"
validation_df = pd.read_csv(validation_path)

print(f"삭제 전 데이터 크기: {validation_df.shape}")
validation_df = validation_df.dropna(subset=["summary"])
print(f"삭제 후 데이터 크기: {validation_df.shape}")

삭제 전 데이터 크기: (47699, 4)
삭제 후 데이터 크기: (47698, 4)
삭제 전 데이터 크기: (5962, 4)
삭제 후 데이터 크기: (5961, 4)


In [ ]:
# csv 파일 저장
output_dir = "/content/drive/MyDrive/응용자연어처리/project/data"

train_df.to_csv(os.path.join(output_dir, "train.csv"), index=False, encoding="utf-8-sig")
validation_df.to_csv(os.path.join(output_dir, "validation.csv"), index=False, encoding="utf-8-sig")

#### contents & summary 길이 분석

In [ ]:
# tokenizer
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v2")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
path = "/content/drive/MyDrive/응용자연어처리/project/data/train.csv"
df = pd.read_csv(path)

df["summary_length"] = df["summary"].apply(lambda x: len(tokenizer(x)["input_ids"]))
print(df["summary_length"].describe())

count    47698.000000
mean        54.673299
std         15.997383
min         10.000000
25%         44.000000
50%         53.000000
75%         64.000000
max        299.000000
Name: summary_length, dtype: float64


In [69]:
df["contents_length"] = df["contents"].apply(lambda x: len(tokenizer(x)["input_ids"]))
print(df["contents_length"].describe())

count    47698.000000
mean       509.780473
std        172.075851
min        156.000000
25%        356.000000
50%        475.000000
75%        664.000000
max       6787.000000
Name: contents_length, dtype: float64
